In [ ]:
# IMPORTANTE:
# não se esqueça de atualizar as datas [filtros e excel]
# sempre a ultima versão do banco analisada

In [1]:
import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")

In [3]:
# puxar arquivo do banco
df = pd.read_csv('eapv_28022022_17h16.csv', sep=';', encoding='utf-8')

print(f'Banco importado: {df.shape}')
banco_completo = df.shape[0]

Banco importado: (17733, 189)


# Excluir as fichas canceladas

In [4]:
# Transformando ['Número da Notificação'] para string
df['Número da Notificação'] = df['Número da Notificação'].astype(str)

In [6]:
# puxar arquivo canceladas
df_canceladas = pd.read_csv('eapv_canceladas_28022022.csv', sep=';', encoding='utf-8')

# df_canceladas passa a ser apenas a coluna 'Número da Notificação' e em string
df_canceladas = df_canceladas[['Número da Notificação']].astype(str)
print(f'Banco canceladas: {df_canceladas.shape}')
banco_canceladas = df_canceladas.shape[0]

# Transformando o banco df_canceladas para uma lista notif_canceladas
notif_canceladas = df_canceladas['Número da Notificação'].to_list()
#notif_canceladas

Banco canceladas: (800, 1)


In [7]:
# TIRANDO AS EXCLUÍDAS DO BANCO DE DADOS 
df = df[~df['Número da Notificação'].str.contains('(?i)' + '|'.join(notif_canceladas), na=False)]

# conferir
print(f'Banco sem as canceladas: {df.shape[0]}')
print(f'Banco completo: {banco_completo} - canceladas: {banco_canceladas} = {banco_completo - banco_canceladas}')

Banco sem as canceladas: 16933
Banco completo: 17733 - canceladas: 800 = 16933


In [9]:
# Verificar se ficou alguma cancelada e excluir do banco
df[df['Situação da Notificação'] == 'Cancelado']
df = df[~df['Situação da Notificação'].str.contains('Cancelado', na=False)]

# Tratar vacinas

In [11]:
df = df.loc[(df['Imunobiológico (vacina)'].str.contains('(?i)Butantan') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Sinovac') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Oxford') |
             df['Imunobiológico (vacina)'].str.contains('(?i)AstraZeneca') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Covishield') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Pfizer') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Janssen'))]

In [12]:
bd_vacinas_covid = df.shape[0]

print(f'Foram excluídas do banco {banco_completo - bd_vacinas_covid} fichas que contém outras vacinas que não as COVID-19')
print(f'Banco somente com vacinas COVID-19: {df.shape}')

Foram excluídas do banco 924 fichas que contém outras vacinas que não as COVID-19
Banco somente com vacinas COVID-19: (16809, 189)


.

.

A PARTIR DAQUI, É POSSÍVEL RODAR SOMENTE AS CÉLULAS DE INTERESSE

# Abaixo de 12 anos

In [43]:
# Encontrar novos casos e enviar para [helena-ramos@saude.rs.gov.br]
df_12_anos = df.copy()

In [44]:
# Filtrar menor que 12 anos
df_12_anos = df_12_anos[df_12_anos['Idade Evento'] < 12]
# Colocar em ordem crescente
df_12_anos = df_12_anos.sort_values('Idade Evento').reset_index(drop=True)
# Dropar valores nulos na coluna 'Nome Completo'
df_12_anos.dropna(subset=['Nome Completo'], inplace=True)

In [45]:
## ALTERAR DATA
# Filtrar apenas as notificaçoes novas
#df_12_anos['Data da Notificação'] = pd.to_datetime(df_12_anos['Data da Notificação'])
#df_12_anos = df_12_anos[df_12_anos['Data da Notificação']>= '2022-02-08']

In [46]:
# Consultar se há novos casos antes de gerar excel
print(f'Há {df_12_anos.shape[0]} casos')
# Gerar excel com nova DATA 
df_12_anos.to_excel('eapv_abaixo_12_anos.xlsx')

Há 367 casos


# Guillain Barre

In [16]:
# Encontrar novos casos e enviar para [rosemeri-cohen@saude.rs.gov.br]
df_guillain = df.copy()

In [ ]:
# pesquisa 'guill' em todo o dataframe
df_guillain = df_guillain.loc[df_guillain.apply(lambda row: row.astype(str).str.contains('(?i)guill').any(), axis=1)]

In [17]:
## ALTERAR DATA
# Filtrar apenas as notificaçoes novas
df_guillain['Data da Notificação'] = pd.to_datetime(df_guillain['Data da Notificação'])
df_guillain = df_guillain[df_guillain['Data da Notificação']>= '2021-12-04']

In [20]:
# Consultar se há novos casos antes de gerar excel
print(f'Há {df_guillain.shape[0]} casos')
# Gerar excel com data recorrente
df_guillain.to_excel('eapv_guillain_barre_07022022.xlsx')

Há 2 casos


# Tromboses

In [ ]:
# Encontrar novos casos e enviar para [rosemeri-cohen@saude.rs.gov.br]
df_tromboses = df.copy()

In [15]:
# Fazer um lista com os nomes para pesquisa 
tromboses = ['trombo', 'embolia']
# Pesquisa {tromboses} em todo o dataframe
df_tromboses = df_tromboses.loc[df_tromboses.apply(lambda row: row.astype(str).str.contains('(?i)' + '|'.join(tromboses)).any(), axis=1)]
# criar coluna para escrever a classificacao manualmente
df_tromboses['Classificacao'] = 'Indefinido'

In [ ]:
## ALTERAR DATA
# Filtrar apenas as notificaçoes novas
df_tromboses['Data da Notificação'] = pd.to_datetime(df_tromboses['Data da Notificação'])
df_tromboses = df_tromboses[df_tromboses['Data da Notificação']>= '2021-12-04']

In [19]:
# Consultar se há novos casos antes de gerar excel
print(f'Há {df_tromboses.shape[0]} casos')
# Gerar excel com data recorrente
#df_tromboses.to_excel('eapv_tromboses_07022022.xlsx')

Há 26 casos


# Gestantes

In [ ]:
# Encontrar novos casos e enviar para [helena-ramos@saude.rs.gov.br]
df_gestantes = df.copy()

In [21]:
df_gestantes = df_gestantes.loc[df_gestantes['Gestante no momento da vacinação?'].str.contains("(?i)Sim", na=False)]
df_gestantes = df_gestantes.loc[df_gestantes['Classificação de gravidade'].str.contains("Grave", na=False)]

In [ ]:
# Filtrar apenas as notificaçoes novas / SEMPRE ALTERAR A DATA
df_gestantes['Data da Notificação'] = pd.to_datetime(df_gestantes['Data da Notificação'])
df_gestantes = df_gestantes[df_gestantes['Data da Notificação']>= '2021-12-04']

In [22]:
# Consultar se há novos casos antes de gerar excel
print(f'Há {df_gestantes.shape[0]} casos')
# Gerar excel com data recorrente
df_gestantes.to_excel('eapv_gestantes_graves_07022022.xlsx')

Há 1 casos


# Encerrados graves por municípios

In [ ]:
# bug no e-sus, mostra encerrados pelo estado na coluna dos municípios... verificar manualmente

In [16]:
# Encontrar novos casos e enviar para [helena-ramos@saude.rs.gov.br]
df_municipio = df.copy()

In [17]:
df_municipio = df_municipio.loc[df_municipio['Classificação de gravidade'].str.contains("Grave", na=False)]
df_municipio.dropna(subset=['Encerramento Municipal - Evento adverso'], inplace=True)

In [ ]:
# Filtrar apenas as notificaçoes novas / SEMPRE ALTERAR A DATA
df_municipio['Data da Notificação'] = pd.to_datetime(df_municipio['Data da Notificação'])
df_municipio = df_municipio[df_municipio['Data da Notificação']>= '2021-12-04']

In [19]:
# Consultar se há novos casos antes de gerar excel
print(f'Há {df_municipio.shape[0]} casos')
# Gerar excel com data recorrente
df_municipio.to_excel('encerrados_por_municipio.xlsx')

Há 1 casos


In [ ]:
# PARA FAZER
# Filtrar apenas colunas de interesse. Identificar quais...
# Verificar casos de SARC em Imunobiológico(vacina) e gerar xlsx